In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime as dt
import os
import pandas as pd
import argparse
from argparse import RawTextHelpFormatter

DATA_DIR = os.path.dirname(os.path.abspath(__file__)).replace("\\", "/") + "/../data/"

NameError: name '__file__' is not defined

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 14 14:25:44 2020

@author: givkriek
"""


# Do not modify this def ! Unless you are an expert ;) 
def forecaster(cons, temp, steps, days, case, name):
    '''
    Forecaster is based on: https://web.archive.org/web/20200319010116if_/https://www.tensorflow.org/tutorials/structured_data/time_series
    '''
        
    # Import dataset in numpy format
    df = pd.read_csv(cons)
    try:
        df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%d/%m/%Y %H:%M:%S")
    except ValueError:
        df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%d/%m/%Y %H:%M")
    df_cons = df.to_numpy()
    
    df = pd.read_csv(temp)
    try:
        df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%d/%m/%Y %H:%M:%S")
    except ValueError:
        df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%d/%m/%Y %H:%M")
    df_temp = df.to_numpy()
    
    # Inputs for neural network  
    dataset = np.zeros((len(df_cons),6))
    for timestep in range(len(df_cons)): 
        # Parameter to forecast
        dataset[timestep,0] = df_cons[timestep][1]
        # Input
        dataset[timestep,1] = df_cons[timestep][0].weekday()
        if df_cons[timestep][0].weekday() <= 4 : 
            dataset[timestep,2] = 0
        else :
            dataset[timestep,2] = 1
        hour = df_cons[timestep][0].hour
        minu = df_cons[timestep][0].minute
        dataset[timestep,3] = int(hour*4+minu/15)
        dataset[timestep,4] = df_cons[timestep][0].month
        dataset[timestep,5] = df_temp[timestep][1]

    pos=0
    # Scale values by subtracting the mean and dividing by the standard deviation
    data_mean = dataset.mean(axis=0)
    data_std = dataset.std(axis=0)
    dataset = (dataset-data_mean)/data_std      
    
    if steps[1] == 1: end = len(dataset)
    else: end = len(dataset) - steps[1]

    def multivariate_data(dataset, pos, start_index, end_index, history_size,
                          target_size, step):
        '''
        Divide data in two groups, shifted 
        '''
        # Inizialize lists
        input_data = []
        output_data = []
        
        # Start seperate data
        start_index = start_index #+ history_size
        for i in range(start_index, end_index):
            input_data.append(dataset[range(i-history_size, i, step)])
            output_data.append(dataset[:, pos][i:i+target_size])
        
        return np.array(input_data), np.array(output_data)

    # Split data
    for i in range(len(df_cons)):
        if df_cons[i][0] == days[3]:
            index = i

    x_train, y_train = multivariate_data(dataset,pos, 
                                         0, days[0]*steps[0], 
                                         steps[0], steps[1], steps[2])
    x_val, y_val = multivariate_data(dataset, pos,
                                     days[0]*steps[0], (days[0] + days[1])*steps[0],
                                     steps[0], steps[1], steps[2])
    x_test, y_test = multivariate_data(dataset, pos,
                                       index, end, 
                                       steps[0], steps[1], steps[2])
    
    # Special parameters to tune data
    BATCH_SIZE = 60 
    BUFFER_SIZE = 10000 
    
    # Shuffle, batch and slice data
    tf.keras.backend.set_floatx('float64')
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE).repeat()

        
    def training(steps, train_data, val_data):
        '''
        The training of the neural network
        '''
        # Setting seed to ensure reproducibility. 
        tf.random.set_seed(3)  
        
        # Special parameters to tune training
        EPOCHS = 10
        hidden_layers = 20
        STEPS = 100
        OPTIMIZATION = 'mean_squared_error'  
    
        # Create model
        model = tf.keras.models.Sequential()
        # Create layers
        model.add(tf.keras.layers.LSTM(hidden_layers))  # Set number of hidden layers
        model.add(tf.keras.layers.Dense(steps[1], activation = 'elu'))  # Dense = number of output layers
        # Set objective function
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.000764), loss = OPTIMIZATION)
        
        # Train
        history = model.fit(train_data, 
                            epochs = EPOCHS,
                            steps_per_epoch = STEPS, 
                            validation_data = val_data,
                            validation_steps = STEPS) 
              
        print('\n')
        print('Training is done')
        print('\n')
        
        return history, model

    def save_model(model, name):
        '''
        Once the neural network is trained, and it is used for several simulations,
        it possible to save the training, to avoid re-training every time
        '''
        model.save(name+".h5")
        
    def load_model(name):
        '''
        if model is saved somewhere, and user wants to get it back, he
        should use this funtion
        '''
        model = tf.keras.models.load_model(name+".h5")
        
        return model
    
    def plot_train_history(history):
        '''
        Convergence plots to have an idea on how the training performs
        '''
        print('\n')
        print('Convergence plot is diplaying ... To continue -> close the graph')
        print('\n')
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        
        epochs = range(len(loss))
        
        plt.figure()
        plt.plot(epochs, loss, 'b', label='Training loss')
        plt.plot(epochs, val_loss, 'r', label='Validation loss')
        plt.title('Training and validation losses')
        plt.legend()
        plt.show()  
    
    def test_forecaster(data):
        '''
        Only use this function when neural network is trained
        '''
        print('\n')
        print('Neural network test starts !')
        print('\n')
        true_future = np.array([])
        predicted_future = np.array([])
        
        [days, steps, pos, model, x_test, y_test, data_std, data_mean] = data
        days_to_test = days[2]
        
        for step in range(int(days_to_test*(steps[0]/steps[1]))):
            y = (y_test[step*steps[1]:step*steps[1] + steps[1], pos] * data_std[pos]) + data_mean[pos]  
            predict_future = (np.array(model.predict(x_test[step*steps[1]:step*steps[1] + steps[0]])[pos])* data_std[pos]) + data_mean[pos]
            
            true_future = np.append(true_future, y)
            predicted_future = np.append(predicted_future, predict_future)
            print('Day number',step,'has been predicted')
        return [true_future, predicted_future]
            
                
    def plot_results(results):
        '''
        Once the neural network is tested, this function plots the results
        '''
        the_real_demand = results[0]
        the_forecasted_demand = results[1]
        print('Plotting results ... To continue -> close the graph')
        x = np.arange(len(the_real_demand))
        plt.figure()
        plt.xlabel('Time')
        plt.ylabel('Demand [kW]')
        plt.step(x,the_real_demand, label='Real demand')
        plt.step(x,the_forecasted_demand, label='Forecasted demand')
        plt.legend()
        plt.show()
        
    def validation(forecasted, real, parameter):
        ''' 
        compute some important parameters to compare forecasting results
        '''
        value = 0
        value_1 = 0
        value_2 = 0
        
        if parameter == 'SMAPE':
            for i in range(len(forecasted)):
                if real[i] + forecasted[i] == 0:
                    value += 0
                else: 
                    value += ((abs(real[i] - forecasted[i])) / (real[i] + forecasted[i])) * 100
            final_value = value / len(forecasted)  
            
        elif parameter == 'MAPE':
            for i in range(len(forecasted)):
                if real[i] == 0:
                    value += 0
                else: 
                    value += (abs(real[i] - forecasted[i]))/real[i]
            final_value = value / len(forecasted) * 100
            
        elif parameter == 'RMSE':
            for i in range(len(forecasted)):
                value += (real[i] - forecasted[i]) ** 2
            final_value = (value / len(forecasted)) ** (1 / 2) 
            
        elif parameter == 'R':
            for i in range(len(forecasted)):
                value += (real[i] - np.mean(real)) * (forecasted[i] - np.mean(forecasted))
                value_1 += (real[i] - np.mean(real)) ** 2
                value_2 += (forecasted[i] - np.mean(forecasted)) ** 2
    
            if value_1 == 0 or value_2 == 0:
                final_value = 100
            else:
                final_value = (value / ((value_1 ** (1 / 2)) * (value_2 ** (1 / 2))))*100
            
        return final_value

        
    def create_csv(days, initial_day):
        '''
        Receive info to export in csv
        For now: Date, time and power
        '''       

        dates = []
        for i in range(len(results[1])):
            dates.append(initial_day)
            initial_day += dt.timedelta(minutes=15)
            
        dataset = [dates,results[1]]
        
        with open('forecasted_demand.csv', 'w') as f:
            f.write('Date,"Forecasted consumption kW"')
            f.write("\n")
            for i in range(len(dataset[0])):
                f.write(str(dataset[0][i]) + ',' + str(dataset[1][i]))
                f.write("\n")
                
    # Start process
    if case[0]:
        history, model = training(steps, train_data, val_data)
        save_model(model, name)
        plot_train_history(history)
        
    else:
        model = load_model(name)
    
    
    # Concatenate data
    data = [days, steps, pos, model, x_test, y_test, data_std, data_mean] 
    # The actual forecasting
    results = test_forecaster(data)
    # Plot results
    plot_results(results)
    # Print score of forecaster (RMSE, etc)
    validation(results[1],results[0], 'MAPE')
    # Export to csv (located in results files)
    if case[1]:
        create_csv(days, days[-1])
        
    
def configuration():
    '''
    configuration
    '''
    epilogue_usage = """ Complete example: python forecast.py -tt 96 -T 200,20 -N my_model -t 10 -f 2016/12/01 -e 0  """
    
    parser = argparse.ArgumentParser(description='The following arguments must be included. Read the example to understand how it works', epilog= epilogue_usage,formatter_class=RawTextHelpFormatter)
    
    parser.add_argument('-tt', '--timesteps', required=True, 
                        help='How many timesteps to forecast ? Day-ahead=96, 12h-ahead=48, etc', default=False)
    parser.add_argument('-T', '--training', required=True, 
                        help='Train neural network? NO ? enter 0. YES ? Enter number of days to train and validate with a coma to differentiate (e.g. 200,10)', default=False)
    parser.add_argument('-N', '--name', required=True, 
                        help='Name your neural network to train or to load', default=False)
    parser.add_argument('-t', '--test', required=True, 
                        help='Give the number of days to test', default=False)
    parser.add_argument('-f', '--first_day', required=True, 
                        help='Give first day to test in format:  yyyy/mm/dd', default=False)
    parser.add_argument('-e', '--export', required=True, 
                        help='Save test results? Yes, write 1, no then write 0', default=False)

    args = parser.parse_args()
    
    print('\n')
    print('Number of timesteps to forecast is: ',args.timesteps)
    print('\n')
    # TRAIN
    if len(args.training) > 0:
        pos = args.training.find(',')
        days = [int(args.training[:pos]), int(args.training[pos+1:])]
        case = [True]
        print('You decided to train a neural network with',days[0],'days and to validate it with',days[1],'days')
        print('The neural network will be trained and saved under the name :',args.name)
        
    else: 
        case = [False]
        print('You decided to use a trained neural network under name of:',args.name)
    
    # Name of neural network
    name = args.name
    
    # TEST
    days.append(int(args.test))
    days.append(dt.datetime.strptime(args.first_day,"%Y/%m/%d"))
    print('\n')
    print('The number of days to test is:', str(days[2]), 'with as first day to test:',str(days[3]))
    print('\n')
    if case[0] : 
        print('Training starts !')
        print('\n')

    # Export results in csv format ?
    if int(args.export): case.append(True)
    else: case.append(False)
        
    # Find path with data info
    DATA_DIR = os.path.dirname(os.path.abspath(__file__)) + "/../../data/"   
    cons = DATA_DIR+'dieteren_case/environment/Consumption_15min.csv'
    temp = DATA_DIR+'dieteren_case/environment/Temp_15min.csv'
    
    # Configuration
    steps = [96,int(args.timesteps),1]
    
    forecaster(cons, temp, steps, days, case, name)
    
    print('\n')
    print('The end')
    print('\n')
    
if __name__ == "__main__":
    configuration()
    
    
    

NameError: name 'argparse' is not defined